# Tabu Search per Problema delle N Regine 

# Import

In [1]:
import random
import numpy as np
import copy

## Inizializzazione stato

In [2]:
def inizializza(sol):
    
    # shake shake shake
    for c in range(0,DIMENSIONE-1):
        sol = tweak(sol)
    return sol


def tweak(sol):
    
    sol_copy = np.copy(sol)
    
    # scegli random due colonne distinte
    x = random.randint(0,DIMENSIONE-1)
    y = random.randint(0,DIMENSIONE-1)
    while x==y:
        y = random.randint(0,DIMENSIONE-1)
        
    # scambia le due colonne
    temp = sol_copy[y]
    sol_copy[y] = sol_copy[x] 
    sol_copy[x] = temp
    
    return sol_copy

## Generazione Successori di uno Stato

In [3]:
def generazione_successori(stato): 
    """"
    genera la lista ordinata di successori di uno stato
    """""
        
    lista = []
    t = len(stato)
    
    for i in range(0, t-1):
        for j in range(i+1, t):
            buffer = copy.deepcopy(stato)
            temp = buffer[i]
            buffer[i] = buffer[j]
            buffer[j] = temp
            eval_successore = eval_stato(buffer)
            lista.append((buffer, eval_successore, (stato[i], stato[j])))  

    lista.sort(key=lambda x: x[1])  # ordiniamo i successori in base alla loro valutazione eval_stato
    return(lista)

## Funzione per la Valutazione di uno Stato (calcolo attacchi)

In [4]:
def eval_stato(stato):
    
    # definizione della scacchiera N x N
    board  = [[0] * DIMENSIONE for i in range(DIMENSIONE)] 
    
    # inserimento delle regine ('Q') nelle loro posizioni sulla scacchiera
    for i in range(0,DIMENSIONE):
        board[stato[i]][i]='Q'
        
    # spostamenti possibili sulla scacchiera
    dx = [-1,1,-1,1]
    dy = [-1,1,1,-1]
    
    # inizializzazione numero di attacchi (diretti o indiretti)
    conflitti = 0

    for i in range(0,DIMENSIONE):
        x=stato[i]
        y=i
        
        # verifica attacchi sulle diagonali       
        for j in range(0,4):
            tempx = x
            tempy = y
            while (True):
                tempx = tempx + dx[j]           # spostamento sull'asse x
                tempy = tempy + dy[j]           # spostamento sull'asse y
                if ((tempx < 0) or 
                    (tempx >= DIMENSIONE) or
                    (tempy < 0) or 
                    (tempy >= DIMENSIONE)):
                    break                       # si esce dal ciclo while se lo spostamento va fuori la scacchiera
                if (board[tempx][tempy]=='Q'):
                    conflitti = conflitti + 1   # aggiornamento numero di attacchi
    return conflitti

## Test Assenza Mossa nella Lista Tabu

In [5]:
def tabu_test(sequenza, tabu_list):    # è True se una mossa NON è presente nella tabu_list
    
    a, b = sequenza[2]
    if ((a, b) in tabu_list or (b, a) in tabu_list):
        assente = False
    else:
        assente = True
    return(assente)

## Stampa scacchiera

In [6]:
def stampa(sol):
    
    board = [[0] * DIMENSIONE for i in range(DIMENSIONE)] 

    for x in range(0,DIMENSIONE):
        board[sol[x]][x]='Q'
    print("SCACCHIERA")
    for x in range(0,DIMENSIONE):
        for y in range(0,DIMENSIONE):
            if(board[x][y]=='Q'):
                print("Q   ",end=''),
            else:
                print(".   ",end=''),
        print("\n")
    print("\n\n")

## Algoritmo Tabu Search

In [7]:
def tabu_search(tabu_tenure):
    
    # impostazione stato iniziale
    stato = list(x for x in range(DIMENSIONE))   
    current = inizializza(stato)
    eval_current = eval_stato(current)
    
    # inizializzazione best
    best = current
    eval_best = eval_current
        
    tabu_list = {}
    neighbours = []
        
    cont = 0
    
    # while not criterio_terminazione():
    while (cont < 100 and eval_best != 0):
            
        # generazione successori (stato, eval_stato, mossa) e ordinamento su eval_stato
        lista_successori = generazione_successori(current)
        if cont == 0:
            l = len(lista_successori)
            print('Numero successori: ', l, '\n')
            
        # selezione successori non tabu
        neighbours = list(filter(lambda n: tabu_test(n, tabu_list), lista_successori))
            
        next_state = neighbours[0][0]         # selezione del migliore dei successori
        eval_next_state = neighbours[0][1]
        print("Iterazione ", cont, ':')
        print('next_state: ', eval_next_state)
        delta = eval_best - eval_next_state
        if delta > 0:
            best = next_state                 # aggiornamento di best
            eval_best = eval_next_state       
        current = next_state
        eval_current = eval_next_state
            
        # decremento del tabu_tenure
        for mossa in tabu_list:
            tabu_list[mossa] = tabu_list[mossa] - 1
                
        # eliminazione elementi con tenure uguale a zero 
        tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k]!=0}   
  
        # inserimento della mossa di next nella tabu_list
        mossa_next = neighbours[0][2]
        tabu_list[mossa_next] = tabu_tenure
        
        print("best_eval =", eval_best,)
        print('mossa:', mossa_next)
        print('tabu_list:', tabu_list, '\n') 
            
        cont += 1
                
    return(best, eval_best)

In [8]:
DIMENSIONE = 30   # dimensione dei lati della scacchiera N x N (dove N è la DIMENSIONE)

In [9]:
soluzione, conflitti = tabu_search(5)

Numero successori:  435 

Iterazione  0 :
next_state:  46
best_eval = 46
mossa: (0, 16)
tabu_list: {(0, 16): 5} 

Iterazione  1 :
next_state:  32
best_eval = 32
mossa: (3, 9)
tabu_list: {(0, 16): 4, (3, 9): 5} 

Iterazione  2 :
next_state:  20
best_eval = 20
mossa: (17, 28)
tabu_list: {(0, 16): 3, (3, 9): 4, (17, 28): 5} 

Iterazione  3 :
next_state:  14
best_eval = 14
mossa: (4, 19)
tabu_list: {(0, 16): 2, (3, 9): 3, (17, 28): 4, (4, 19): 5} 

Iterazione  4 :
next_state:  10
best_eval = 10
mossa: (19, 24)
tabu_list: {(0, 16): 1, (3, 9): 2, (17, 28): 3, (4, 19): 4, (19, 24): 5} 

Iterazione  5 :
next_state:  6
best_eval = 6
mossa: (15, 26)
tabu_list: {(3, 9): 1, (17, 28): 2, (4, 19): 3, (19, 24): 4, (15, 26): 5} 

Iterazione  6 :
next_state:  4
best_eval = 4
mossa: (6, 10)
tabu_list: {(17, 28): 1, (4, 19): 2, (19, 24): 3, (15, 26): 4, (6, 10): 5} 

Iterazione  7 :
next_state:  2
best_eval = 2
mossa: (16, 26)
tabu_list: {(4, 19): 1, (19, 24): 2, (15, 26): 3, (6, 10): 4, (16, 26): 5} 

I

In [10]:
soluzione

array([26, 10, 12,  9, 24, 18,  3,  6, 25, 23, 14, 16,  8,  2, 15,  7,  0,
       28, 20, 27,  1,  4, 22, 13, 17,  5, 29, 21, 19, 11])

In [11]:
conflitti

0

In [12]:
stampa(soluzione)

SCACCHIERA
.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   

.   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   .   .   .   .   .   .   Q   .   .   .   .   .   .   .   .   .   .   .   .   .   .   

.   .   .   .